# Exploratory Data Analysis

# Import the library

In [1]:
import os
import seaborn as sns
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load dataset

In [2]:
current_path = Path.cwd().resolve()
ROOT_DIR = current_path.parents[0]  # Get the parent directory of the current path
print(f"Root directory: {ROOT_DIR}")

DATA_DIR = ROOT_DIR / 'data'
print(f"Data directory: {DATA_DIR}")

TEST_DIR = ROOT_DIR / 'test'
print(f"Test directory: {TEST_DIR}")

Root directory: C:\Users\hongquan\Documents\GitHub\regression_comparison
Data directory: C:\Users\hongquan\Documents\GitHub\regression_comparison\data
Test directory: C:\Users\hongquan\Documents\GitHub\regression_comparison\test


In [3]:
training_data = pd.read_csv(DATA_DIR / 'train.csv')

## Description of the shape and type of the dataset

In [4]:
# Show shape of the dataset
print(f"Shape of the dataset: {training_data.shape}")

Shape of the dataset: (1460, 81)


In [5]:
# Show the columns of the dataset
print(f"Columns of the dataset: {training_data.columns.tolist()}")

Columns of the dataset: ['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorc

In [6]:
# Show the first few rows of the dataset
print(f"First few rows of the dataset: ")
training_data.head()

First few rows of the dataset: 


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
# Show the description of the dataset
print(f"Description of the dataset: ")
training_data.describe().T

Description of the dataset: 


,count,mean,std,min,25%,50%,75%,max
Id,1460.0,730.500000,421.610009,1.0,365.75,730.5,1095.25,1460.0
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
LotFrontage,1201.0,70.049958,24.284752,21.0,59.00,69.0,80.00,313.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
MasVnrArea,1452.0,103.685262,181.066207,0.0,0.00,0.0,166.00,1600.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0


In [8]:
# Drop the Id column
training_data = training_data.drop(columns=['Id']) 

In [9]:
# Show information about the dataset
print(f"Information about the dataset: ")
training_data.info() 

Information about the dataset: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 

In [10]:
# Select unique values for object columns
object_df = training_data.select_dtypes(include='object')
object_cols = object_df.columns.unique().to_list()
print(f"Lenght of object columns: {len(object_cols)}")
print(f"Object columns and their unique values:")
object_df.nunique().to_dict()


Lenght of object columns: 43
Object columns and their unique values:


{'MSZoning': 5,
 'Street': 2,
 'Alley': 2,
 'LotShape': 4,
 'LandContour': 4,
 'Utilities': 2,
 'LotConfig': 5,
 'LandSlope': 3,
 'Neighborhood': 25,
 'Condition1': 9,
 'Condition2': 8,
 'BldgType': 5,
 'HouseStyle': 8,
 'RoofStyle': 6,
 'RoofMatl': 8,
 'Exterior1st': 15,
 'Exterior2nd': 16,
 'MasVnrType': 3,
 'ExterQual': 4,
 'ExterCond': 5,
 'Foundation': 6,
 'BsmtQual': 4,
 'BsmtCond': 4,
 'BsmtExposure': 4,
 'BsmtFinType1': 6,
 'BsmtFinType2': 6,
 'Heating': 6,
 'HeatingQC': 5,
 'CentralAir': 2,
 'Electrical': 5,
 'KitchenQual': 4,
 'Functional': 7,
 'FireplaceQu': 5,
 'GarageType': 6,
 'GarageFinish': 3,
 'GarageQual': 5,
 'GarageCond': 5,
 'PavedDrive': 3,
 'PoolQC': 3,
 'Fence': 4,
 'MiscFeature': 4,
 'SaleType': 9,
 'SaleCondition': 6}

In [11]:
# select unique values for non-object columns
non_object_df = training_data.select_dtypes(exclude='object')
non_object_cols = non_object_df.columns.unique().to_list()
print(f"Lenght of non-object columns: {len(non_object_cols)}")
print(f"Non-object columns and their unique values:")
non_object_df.nunique().to_dict()

Lenght of non-object columns: 37
Non-object columns and their unique values:


{'MSSubClass': 15,
 'LotFrontage': 110,
 'LotArea': 1073,
 'OverallQual': 10,
 'OverallCond': 9,
 'YearBuilt': 112,
 'YearRemodAdd': 61,
 'MasVnrArea': 327,
 'BsmtFinSF1': 637,
 'BsmtFinSF2': 144,
 'BsmtUnfSF': 780,
 'TotalBsmtSF': 721,
 '1stFlrSF': 753,
 '2ndFlrSF': 417,
 'LowQualFinSF': 24,
 'GrLivArea': 861,
 'BsmtFullBath': 4,
 'BsmtHalfBath': 3,
 'FullBath': 4,
 'HalfBath': 3,
 'BedroomAbvGr': 8,
 'KitchenAbvGr': 4,
 'TotRmsAbvGrd': 12,
 'Fireplaces': 4,
 'GarageYrBlt': 97,
 'GarageCars': 5,
 'GarageArea': 441,
 'WoodDeckSF': 274,
 'OpenPorchSF': 202,
 'EnclosedPorch': 120,
 '3SsnPorch': 20,
 'ScreenPorch': 76,
 'PoolArea': 8,
 'MiscVal': 21,
 'MoSold': 12,
 'YrSold': 5,
 'SalePrice': 663}

# Descriptive Statistic

In [12]:
# Use plotly to visualize the distribution of SalePrice
px.histogram(data_frame= training_data, x= 'SalePrice', title='Distribution of SalePrice')

In [13]:
# Plot the violin plot for SalePrice
px.violin(training_data, y='SalePrice', box=True, points='all', title='Distribution of SalePrice', width=800, height=800)

In [14]:
# Calculate the correlation matrix for non-object columns
coor_matrix = training_data[non_object_cols].corr()
coor_matrix

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
MSSubClass,1.000000,-0.386347,-0.139781,0.032628,-0.059316,0.027850,0.040581,0.022936,-0.069836,-0.065649,...,-0.012579,-0.006100,-0.012037,-0.043825,-0.026030,0.008283,-0.007683,-0.013585,-0.021407,-0.084284
LotFrontage,-0.386347,1.000000,0.426095,0.251646,-0.059213,0.123349,0.088866,0.193458,0.233633,0.049900,...,0.088521,0.151972,0.010700,0.070029,0.041383,0.206167,0.003368,0.011200,0.007450,0.351799
LotArea,-0.139781,0.426095,1.000000,0.105806,-0.005636,0.014228,0.013788,0.104160,0.214103,0.111170,...,0.171698,0.084774,-0.018340,0.020423,0.043160,0.077672,0.038068,0.001205,-0.014261,0.263843
OverallQual,0.032628,0.251646,0.105806,1.000000,-0.091932,0.572323,0.550684,0.411876,0.239666,-0.059119,...,0.238923,0.308819,-0.113937,0.030371,0.064886,0.065166,-0.031406,0.070815,-0.027347,0.790982
OverallCond,-0.059316,-0.059213,-0.005636,-0.091932,1.000000,-0.375983,0.073741,-0.128101,-0.046231,0.040229,...,-0.003334,-0.032589,0.070356,0.025504,0.054811,-0.001985,0.068777,-0.003511,0.043950,-0.077856
YearBuilt,0.027850,0.123349,0.014228,0.572323,-0.375983,1.000000,0.592855,0.315707,0.249503,-0.049107,...,0.224880,0.188686,-0.387268,0.031355,-0.050364,0.004950,-0.034383,0.012398,-0.013618,0.522897
YearRemodAdd,0.040581,0.088866,0.013788,0.550684,0.073741,0.592855,1.000000,0.179618,0.128451,-0.067759,...,0.205726,0.226298,-0.193919,0.045286,-0.038740,0.005829,-0.010286,0.021490,0.035743,0.507101
MasVnrArea,0.022936,0.193458,0.104160,0.411876,-0.128101,0.315707,0.179618,1.000000,0.264736,-0.072319,...,0.159718,0.125703,-0.110204,0.018796,0.061466,0.011723,-0.029815,-0.005965,-0.008201,0.477493
BsmtFinSF1,-0.069836,0.233633,0.214103,0.239666,-0.046231,0.249503,0.128451,0.264736,1.000000,-0.050117,...,0.204306,0.111761,-0.102303,0.026451,0.062021,0.140491,0.003571,-0.015727,0.014359,0.386420
BsmtFinSF2,-0.065649,0.049900,0.111170,-0.059119,0.040229,-0.049107,-0.067759,-0.072319,-0.050117,1.000000,...,0.067898,0.003093,0.036543,-0.029993,0.088871,0.041709,0.004940,-0.015211,0.031706,-0.011378


In [15]:
# Skewness describes the shape of a distribution and tells us if the data is symmetrical or not.
# Skewness = 0 → symmetrical
# Skewness > 0 → right (positive) skew
# Skewness < 0 → left (negative) skew
# Get the skweness of the dataset
skewness = training_data[non_object_cols].skew().sort_values(ascending=False)
print(f"Skewness of the dataset: {skewness}")
print('\n')
print(f"5 highest skewness values: \n{skewness.head(5)}")

Skewness of the dataset: MiscVal          24.476794
PoolArea         14.828374
LotArea          12.207688
3SsnPorch        10.304342
LowQualFinSF      9.011341
KitchenAbvGr      4.488397
BsmtFinSF2        4.255261
ScreenPorch       4.122214
BsmtHalfBath      4.103403
EnclosedPorch     3.089872
MasVnrArea        2.669084
OpenPorchSF       2.364342
LotFrontage       2.163569
SalePrice         1.882876
BsmtFinSF1        1.685503
WoodDeckSF        1.541376
TotalBsmtSF       1.524255
MSSubClass        1.407657
1stFlrSF          1.376757
GrLivArea         1.366560
BsmtUnfSF         0.920268
2ndFlrSF          0.813030
OverallCond       0.693067
TotRmsAbvGrd      0.676341
HalfBath          0.675897
Fireplaces        0.649565
BsmtFullBath      0.596067
OverallQual       0.216944
MoSold            0.212053
BedroomAbvGr      0.211790
GarageArea        0.179981
YrSold            0.096269
FullBath          0.036562
GarageCars       -0.342549
YearRemodAdd     -0.503562
YearBuilt        -0.613461
Gar

In [16]:
# Kurtosis ≈ 0: The distribution has a sharpness similar to the normal distribution (called mesokurtic).
# Kurtosis > 0: The distribution is more peaked than normal, with a higher peak and heavier tails (leptokurtic).
# → Data is highly concentrated around the mean, but has more extreme values (outliers) in the tails.
# Kurtosis < 0: The distribution is flatter than normal, with a lower peak and thinner tails (platykurtic).
# → Data is more evenly spread out and less concentrated around the center.
# Get the kurtosis of the dataset
kurtosis = training_data[non_object_cols].kurtosis().sort_values(ascending=False)
print(f"Kurtosis of the dataset: {kurtosis}")
print('\n')
print(f"5 highest kurtosis values: \n{kurtosis.head(5)}")

Kurtosis of the dataset: MiscVal          701.003342
PoolArea         223.268499
LotArea          203.243271
3SsnPorch        123.662379
LowQualFinSF      83.234817
KitchenAbvGr      21.532404
BsmtFinSF2        20.113338
ScreenPorch       18.439068
LotFrontage       17.452867
BsmtHalfBath      16.396642
TotalBsmtSF       13.250483
BsmtFinSF1        11.118236
EnclosedPorch     10.430766
MasVnrArea        10.082417
OpenPorchSF        8.490336
SalePrice          6.536282
1stFlrSF           5.745841
GrLivArea          4.895121
WoodDeckSF         2.992951
BedroomAbvGr       2.230875
MSSubClass         1.580188
OverallCond        1.106413
GarageArea         0.917067
TotRmsAbvGrd       0.880762
BsmtUnfSF          0.474994
GarageCars         0.220998
OverallQual        0.096293
Fireplaces        -0.217237
MoSold            -0.404109
GarageYrBlt       -0.418341
YearBuilt         -0.439552
2ndFlrSF          -0.553464
BsmtFullBath      -0.839098
FullBath          -0.857043
HalfBath          -1.07

In [17]:
# Get 10 highest correlated features with SalePrice
list_highest_coor = list(coor_matrix['SalePrice'].abs().sort_values(ascending=False).head(10).to_dict())
print(f"10 highest correlated features with SalePrice: {list_highest_coor}")

# Get coorrelation matrix for the highest correlated features
coor_matrix_highest = training_data[list_highest_coor].corr()

10 highest correlated features with SalePrice: ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt']


In [18]:
# Draw the heatmap using plotly
# Note: Plotly does not have a direct equivalent of seaborn's heatmap, but we can use px.imshow to create a similar effect
fig = px.imshow(
    coor_matrix_highest,
    text_auto='.2f',                # Display correlation values with 2 decimal places
    color_continuous_scale='RdBu_r', # Set color scale
    aspect='auto',               # Adjust aspect ratio
    labels=dict(color="Correlation")
)

# Configure the layout of the heatmap
fig.update_layout(
    title='Correlation Heatmap',
    width=800,
    height=800,
    xaxis=dict(tickangle=-45),      # Rotate x-axis labels 
    yaxis=dict(tickangle=0)
)

# Hiển thị biểu đồ
fig.show()

# Deal with missing values 

In [19]:
def plot_missing_values(df):
    # Draw a heatmap using missing values
    missing_value = df.isnull().sum().sort_values(ascending=False)
    missing_value = missing_value[missing_value > 0]  # Filter out columns with no missing values
    if missing_value.empty:
        print("No missing values found in the dataset.")
        return None, None
    missing_value = missing_value.reset_index()

    # Create a bar chart using plotly
    fig_missing = px.bar(
        missing_value,  # Reshape to 2D array for heatmap
        labels={'index': 'Feature', '0': 'Missing Values'},
        x=missing_value['index'],
        y=missing_value[0],
        color_continuous_scale='Blues',
    ).update_layout(
        title='Missing Values Bar Chart',
        xaxis=dict(tickangle=-45),      # Rotate x-axis labels 
    ).show()

    return missing_value, fig_missing

In [20]:
def plot_missing_values_percentage(df):
    # Calculate the percentage of missing values in each column
    missing_values = df.isnull().sum().sort_values(ascending=False)
    missing_values = missing_values[missing_values > 0]  # Filter out columns with no missing values
    if missing_values.empty:
        print("No missing values found in the dataset.")
        return None, None
    missing_values_percentage = missing_values / len(df) * 100

    # Create a bar chart using plotly
    fig_missing_percentage = px.bar(
        missing_values_percentage,  # Filter out columns with no missing values
        labels={'index': 'Feature', 'value': 'Missing Values Percentage'},
        x=missing_values_percentage.index,
        y=missing_values_percentage.values,
        color_continuous_scale='Blues',
    ).update_layout(
        title='Missing Values Percentage Bar Chart',
        xaxis=dict(tickangle=-45),      # Rotate x-axis labels 
    ).show()

    return missing_values_percentage, fig_missing_percentage

## Categorial columns

In [21]:
plot_missing_values(object_df)

(           index     0
 0         PoolQC  1453
 1    MiscFeature  1406
 2          Alley  1369
 3          Fence  1179
 4     MasVnrType   872
 5    FireplaceQu   690
 6     GarageType    81
 7     GarageCond    81
 8     GarageQual    81
 9   GarageFinish    81
 10  BsmtFinType2    38
 11  BsmtExposure    38
 12  BsmtFinType1    37
 13      BsmtQual    37
 14      BsmtCond    37
 15    Electrical     1,
 None)

In [22]:
plot_missing_values_percentage(object_df)

(PoolQC          99.520548
 MiscFeature     96.301370
 Alley           93.767123
 Fence           80.753425
 MasVnrType      59.726027
 FireplaceQu     47.260274
 GarageType       5.547945
 GarageCond       5.547945
 GarageQual       5.547945
 GarageFinish     5.547945
 BsmtFinType2     2.602740
 BsmtExposure     2.602740
 BsmtFinType1     2.534247
 BsmtQual         2.534247
 BsmtCond         2.534247
 Electrical       0.068493
 dtype: float64,
 None)

In [23]:
cols_to_drop = ['Alley', 'PoolQC', 'Fence', 'MiscFeature']
training_data = training_data.drop(columns=cols_to_drop)

### 1. MasVnrType


In [24]:
# Show sum of missing values of MasVnrType
mas_vnr_type_missing = training_data['MasVnrType'].isnull().sum()
print(f"Missing values in MasVnrType: {mas_vnr_type_missing}")

# Show the unique values of MasVnrType
mas_vnr_type_unique = training_data['MasVnrType'].unique()
print(f"Unique values in MasVnrType: {mas_vnr_type_unique}")

Missing values in MasVnrType: 872
Unique values in MasVnrType: ['BrkFace' nan 'Stone' 'BrkCmn']


In [25]:
# Because the MasVnrType column has a lot of missing values
# The value nan of MasVnrType column likely represents houses
# without masonry veneer, so we can fill it with 'None'
training_data['MasVnrType'] = training_data['MasVnrType'].fillna('None')

### 2. FirePlaceQu

In [26]:
# Show the sum of missing values of FirePlaceQu
fireplace_qu_missing = training_data['FireplaceQu'].isnull().sum()
print(f"Missing values in FirePlaceQu: {fireplace_qu_missing}")

# Show the unique values of FirePlaceQu
fireplace_qu_unique = training_data['FireplaceQu'].unique()
print(f"Unique values in FirePlaceQu: {fireplace_qu_unique}")

Missing values in FirePlaceQu: 690
Unique values in FirePlaceQu: [nan 'TA' 'Gd' 'Fa' 'Ex' 'Po']


In [27]:
# Because the FirePlaceQu column has a lot of missing values
# The value nan of FirePlaceQu column represents fire place in houses 
# So we can fill it with 'None'
training_data['FireplaceQu'] = training_data['FireplaceQu'].fillna('None')

### 3. Garage: GarageType, GarageCond, GarageQual, GarageFinish

In [28]:
# The GarageType, GarageCond, GarageQual, GarageFinish columns have a some missing values
# We can fill them with 'None' because they represent houses without garage
garage_cols = ['GarageType', 'GarageCond', 'GarageQual', 'GarageFinish']
for col in garage_cols:
    training_data[col] = training_data[col].fillna('None')

### 4. Basement:  BsmtFinType2, BsmtExposure, BsmtFinType1, BsmtQual, BsmtCond 

In [29]:
# The basement columns have a some missing values
# We can fill them with 'None' because they represent houses without basement
basement_cols = ['BsmtFinType2', 'BsmtExposure', 'BsmtFinType1', 'BsmtQual', 'BsmtCond']
for col in basement_cols:
    training_data[col] = training_data[col].fillna('None')

### 5. Electrical

In [30]:
# Show the missing values of Electrical
electrical_missing = training_data['Electrical'].isnull().sum()
print(f"Missing values in Electrical: {electrical_missing}")    

# Show the unique values of Electrical
electrical_unique = training_data['Electrical'].unique()
print(f"Unique values in Electrical: {electrical_unique}")

Missing values in Electrical: 1
Unique values in Electrical: ['SBrkr' 'FuseF' 'FuseA' 'FuseP' 'Mix' nan]


In [31]:
# The Electrical column has a just one missing value
# We can fill it with the most common value in the column
most_common_electrical = training_data['Electrical'].mode()[0]
print(f"Most common value in Electrical: {most_common_electrical}")

# Fill the missing value in Electrical with the most common value
training_data['Electrical'] = training_data['Electrical'].fillna(most_common_electrical)

Most common value in Electrical: SBrkr


### Check the missing value of object columns

In [32]:
# Drop the columns that have been removed from object_cols
for i in cols_to_drop:
    try:
        object_cols.remove(i)
    except ValueError:
        print(f"Column {i} not found in object_cols, skipping removal.")

In [33]:
# Check the missing value of object columns
object_df = training_data.select_dtypes(include='object')[object_cols]
plot_missing_values(object_df)


No missing values found in the dataset.


(None, None)

## Numerical columns

In [34]:
plot_missing_values(non_object_df)

(         index    0
 0  LotFrontage  259
 1  GarageYrBlt   81
 2   MasVnrArea    8,
 None)

In [35]:
plot_missing_values_percentage(non_object_df)

(LotFrontage    17.739726
 GarageYrBlt     5.547945
 MasVnrArea      0.547945
 dtype: float64,
 None)

### 1. LotFrontage

In [36]:
# Show the sum of missing values in LotFrontage
lot_frontage_missing = training_data['LotFrontage'].isnull().sum()
print(f"Sum of missing values in LotFrontage: {lot_frontage_missing}")

# Check the unique values in LotFrontage
unique_lot_frontage = training_data['LotFrontage'].unique()
print(f"Unique values in LotFrontage: {unique_lot_frontage}")

Sum of missing values in LotFrontage: 259
Unique values in LotFrontage: [ 65.  80.  68.  60.  84.  85.  75.  nan  51.  50.  70.  91.  72.  66.
 101.  57.  44. 110.  98.  47. 108. 112.  74. 115.  61.  48.  33.  52.
 100.  24.  89.  63.  76.  81.  95.  69.  21.  32.  78. 121. 122.  40.
 105.  73.  77.  64.  94.  34.  90.  55.  88.  82.  71. 120. 107.  92.
 134.  62.  86. 141.  97.  54.  41.  79. 174.  99.  67.  83.  43. 103.
  93.  30. 129. 140.  35.  37. 118.  87. 116. 150. 111.  49.  96.  59.
  36.  56. 102.  58.  38. 109. 130.  53. 137.  45. 106. 104.  42.  39.
 144. 114. 128. 149. 313. 168. 182. 138. 160. 152. 124. 153.  46.]


In [37]:
# plot the violin plot for LotFrontage
px.violin(training_data, y='LotFrontage', box=True, points='all')

In [38]:
# The LotFrontage column has some outliers and missing values.
# We can filled the missing values in LotFrontage with the mean value
mean_lot_frontage = training_data['LotFrontage'].mean()
# Fill the missing values in LotFrontage with the mean value
training_data['LotFrontage'] = training_data['LotFrontage'].fillna(mean_lot_frontage)

### 2. GarageYrBlt

In [39]:
# Get index of nan value of GarageType
index_nan_GarageType = training_data[training_data['GarageType'] == 'None'].index

# Get index of nan value of GaraYrBlt
index_nan_GaraYrBlt = training_data[training_data['GarageYrBlt'].isna()].index

# Compare index of 2 features
index_nan_GarageType == index_nan_GaraYrBlt

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [40]:
# Because the nan index of GarageType and GaraYrBlt is same
# So we can't use median or mean value to fill in the nan value (These homes 
# don't have a garage)
training_data['GarageYrBlt'] = training_data['GarageYrBlt'].fillna('No Garage')

### 3. MasVnrArea

In [41]:
# 1. Check how many rows have missing values (NaN) in 'MasVnrArea'
print("Number of rows with MasVnrArea as NaN:", training_data[training_data['MasVnrArea'].isnull()].shape[0])  # Output: 8

Number of rows with MasVnrArea as NaN: 8


In [42]:
# 2. Display details of these rows to inspect their 'MasVnrType'
print("\nDetails of rows where MasVnrArea is NaN:")
display(training_data[training_data['MasVnrArea'].isnull()][['MasVnrType', 'MasVnrArea']])


Details of rows where MasVnrArea is NaN:


,MasVnrType,MasVnrArea
234,None,NaN
529,None,NaN
650,None,NaN
936,None,NaN
973,None,NaN
977,None,NaN
1243,None,NaN
1278,None,NaN


In [43]:
# 3. Count how many houses have no masonry veneer (MasVnrType == 'None')
print(f"\nNumber of houses with MasVnrType = 'None': {training_data[training_data['MasVnrType'] == 'None'].shape[0]}")  # Output: 872


Number of houses with MasVnrType = 'None': 872


In [44]:
# # 4. Find inconsistent rows: MasVnrType is 'None' but MasVnrArea is not zero
# # This violates logical consistency — if there's no veneer, area should be 0
inconsistent_rows = training_data[(training_data['MasVnrType'] == 'None') & (training_data['MasVnrArea'] != 0)]
print(f"\nNumber of inconsistent rows (MasVnrType='None' but MasVnrArea ≠ 0): {len(inconsistent_rows)}")
inconsistent_rows[['MasVnrType', 'MasVnrArea']]


Number of inconsistent rows (MasVnrType='None' but MasVnrArea ≠ 0): 13


,MasVnrType,MasVnrArea
234,None,NaN
529,None,NaN
624,None,288.0
650,None,NaN
773,None,1.0
936,None,NaN
973,None,NaN
977,None,NaN
1230,None,1.0
1243,None,NaN


In [45]:
# # 5. Check consistent rows: MasVnrType is 'None' and MasVnrArea is 0
# # These are logically correct
consistent_rows = training_data[(training_data['MasVnrType'] == 'None') & (training_data['MasVnrArea'] == 0)]
print(f"Number of consistent rows (MasVnrType='None' and MasVnrArea = 0): {len(consistent_rows)}")

Number of consistent rows (MasVnrType='None' and MasVnrArea = 0): 859


In [46]:
# # 6. FIX INCONSISTENCY: Set MasVnrArea = 0 where MasVnrType is 'None' but area is non-zero
# # Logical rule: No veneer → area must be zero
training_data.loc[(training_data['MasVnrType'] == 'None') & (training_data['MasVnrArea'] != 0), 'MasVnrArea'] = 0

In [47]:
# 7. HANDLE MISSING VALUES in MasVnrArea
# We now handle the 8 NaNs in MasVnrArea based on MasVnrType:
# - If MasVnrType is 'None', then MasVnrArea should be 0
# - Otherwise, impute using group mean (by MasVnrType) for better accuracy

# Step 1: Fill MasVnrArea = 0 if MasVnrType is 'None' and MasVnrArea is NaN
training_data.loc[(training_data['MasVnrType'] == 'None') & (training_data['MasVnrArea'].isnull()), 'MasVnrArea'] = 0

# Step 2: For remaining NaNs, fill with the average MasVnrArea within the same MasVnrType group
# This preserves patterns — e.g., 'BrkFace' homes will use average BrkFace area
training_data['MasVnrArea'] = training_data.groupby('MasVnrType')['MasVnrArea'].transform(lambda x: x.fillna(x.mean()))

# # Step 3: If any NaNs still remain (e.g., a group had all NaNs), fill with overall mean
if training_data['MasVnrArea'].isnull().any():
    training_data['MasVnrArea'] = training_data['MasVnrArea'].fillna(training_data['MasVnrArea'].mean())

# Step 4 Final validation: Ensure no inconsistencies remain
# Double-check that no house has 'None' veneer type but non-zero area
final_check = training_data[(training_data['MasVnrType'] == 'None') & (training_data['MasVnrArea'] != 0)]
print(f"Length of final check: {final_check}")

Length of final check: Empty DataFrame
Columns: [MSSubClass, MSZoning, LotFrontage, LotArea, Street, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, OverallQual, OverallCond, YearBuilt, YearRemodAdd, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, MasVnrArea, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, Heating, HeatingQC, CentralAir, Electrical, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, KitchenQual, TotRmsAbvGrd, Functional, Fireplaces, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageArea, GarageQual, GarageCond, PavedDrive, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal, MoSold, YrSold, SaleType, SaleCondition, SalePrice]
Index: []

[0 rows x 76 columns]
